In [ ]:
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

#### **🛩 Import libraries and dataset**
****

1. Import libraries necessary and dataset

In [ ]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 

from itertools import combinations 
from scipy import stats 

from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder 
from sklearn.model_selection import train_test_split, cross_val_score 
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix

from sklearn.linear_model import LogisticRegression 




    1. Write the code that imports the train and test datasets (hint: use pd.read_csv) and assign it to a variables train and test (hint: Notebook path is '/kaggle/input/ai-craft-hackathon-airline-passenger-satisfaction/test.csv')

In [ ]:
train = train[['id','Gender', 'Age', 'Type of Travel', 'Class',
       'Flight Distance', 'Inflight wifi service',
       'Departure/Arrival time convenient', 'Seat comfort',
       'Checkin service',
       'Cleanliness', 'Departure Delay in Minutes', 'satisfaction']]
test = test[['id','Gender', 'Age', 'Type of Travel', 'Class',
       'Flight Distance', 'Inflight wifi service',
       'Departure/Arrival time convenient', 'Seat comfort',
       'Checkin service',
       'Cleanliness', 'Departure Delay in Minutes']]

In [ ]:
train.head()

2. Create a table that outlines some information on our train data including: Count, unique, frequency, mean, standard deviation etc.

In [ ]:
desc = pd.DataFrame(train.describe(include = 'all').transpose())
desc

4. Use the summary stats function to view important information such as number of missing values, mean value, max value, data type etc

In [ ]:
def summary_stats(df): 
    print(f'The shape of the data is: {df.shape}')
    summary = pd.DataFrame(df.dtypes, columns = ['data type'])
    summary['Number of missing values'] = df.isnull().sum().values 
    summary['% of missing values'] = df.isnull().sum().values / len(df) * 100 
    summary['min value'] = desc['min'].values
    summary['mean value'] = desc['mean'].values
    summary['max value'] = desc['max'].values
    summary['mode value'] = df.mode().values[0]
    return summary


    1. Run the above function of the train dataset

5. Look at those variables that are most corrleated with whether someone is satisfied or not

    1. Gather a list of the categories that are numerical (int64) and those that are categorical (object)

    2. make a copy of our train dataset using train.copy() and assign it to a variable called train_display

    3. Label encode the categorical columns 
        a. To do this assign the object LabelEncoder() to the variable le

        b. use a for loop to re-assign each categorical column train_display[col] to its label encoded version

The below code should now output a list of each category and its correlation with satisfaction

In [ ]:
train_display.corr().loc['satisfaction'].abs().sort_values(ascending=False)

#### **🛩 Exploratory data analysis**
****

6. Create some count plots of non-numerical categorical columns to view how many of each unique valeue that there is? (q: what insights can we get from this?)

In [ ]:
fig, axes = plt.subplots(1, 4, figsize=(15, 5))
for i, column in enumerate(categorical_columns.columns):
    ax = axes[i]
    sns.countplot(data=categorical_columns, x=column, ax=ax)
    ax.set_title(column)
    ax.tick_params(axis='x', rotation=45)

    
plt.tight_layout()
plt.show()

    1. Fill in the blanks below

<div class="alert alert-block alert-info" style="background-color: #C2F3D3; color: black;">
- There is an equal number of _____ and ______ participants in the survey. </br>
- Most customers on this airline are _________ travel customers and most of them purchase tickets for ________ class, followed by Eco class. </br>
- A large proportion of customers felt neutral or dissatisfied with the airline.
</div>

7. Create some count plots of numerical categorical columns to view how many of each unique valeue that there is? (q: what insights can we get from this?)

    1. Create a list of all of the columns that are numerical but have only a few unique values, you can do this by running 'train[col].unique()' for all columns where 'col' is the column name

In [ ]:
fig, axes = plt.subplots(1, 5, figsize=(15, 5))
for i, column in enumerate(categoric):
    col = i % 5
    
    ax = axes[col]
    sns.countplot(data=train, x=column, ax=ax)
    
plt.tight_layout()
plt.show()

8. Create some Kernel density plots for view the distribution of continuous variables

    1. Find the numerical columns that are continuous, you can do this by running 'train[col].unique()' for all columns where 'col' is the column name

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(15, 5))
for i, column in enumerate(numeric):
    row = i // 2
    col = i % 2
    
    ax = axes[row, col]
    sns.kdeplot(data=train, x=column, ax=ax)

plt.tight_layout()
plt.show()

    2. Fill in the gaps

<div class="alert alert-block alert-info" style="background-color: #C2F3D3; color: black;">
The variables ___________ and ______________ are all heavily right-skewed. 
</div>

#### **🛩 Machine learning preparation**
****

In [ ]:
clean_train = train.copy()

    1. Fill in the gaps

9. Finding the logarithm can help in the following ways:
    - Normalize the data: Transformation can help in reducing the impact of e_________ values and make the data conform more closely to a _________ distribution.

    - Linearize relationships: In some cases, taking the logarithm of a variable can help linearize relationships between variables, making them more suitable for linear __________ or other linear-based models.

    - Reduce skewness: Skewed data can violate the assumptions of certain models. By applying the logarithm transformation, you can reduce the skewness and make the data more s__________.

    2. From the graphs in section 8, make a list of the categories that were skewed and assign them to a variable called right_skewed

In [ ]:
# Normalize the numeric variables 
for x in right_skewed:
    clean_train[x] = np.log(clean_train[x])

clean_train

In [ ]:
numeric = ["Age", "Flight Distance", "Departure Delay in Minutes"]

fig, axes = plt.subplots(2, 2, figsize=(15, 5))
for i, column in enumerate(numeric):
    row = i // 2
    col = i % 2
    
    ax = axes[row, col]
    sns.kdeplot(data= clean_train, x=column, ax=ax)
    
plt.tight_layout()
plt.show()

    1. Fill in the gaps

<div class="alert alert-block alert-info" style="background-color: #C2F3D3; color: black;">
______________ and _____________ are now more normally distributed. 
</div>

In [ ]:
clean_train.head() 

    1.Define the X and Y variables (hint: x is all categories apart from the id and the one that we are trying to predict and the y variables is only the column that we are trying to predict)

9. We next need to normalize the numeric variables, this is the process of transforming the variables to a common scale or range to ensure a fair comparison between variables and prevent one variable from dominating others due to its larger magnitude, we will normalize using the standard scalar object

    1. Assign a variable named scaler to the object 'StandardScaler()'


In [ ]:

numeric_col = ["Age", "Flight Distance", "Departure Delay in Minutes"]

# Handle the problematic values in X
for numeric_val in numeric_col:
    X[numeric_val] = X[numeric_val].replace([np.inf, -np.inf], np.nan)
    X[numeric_val] = X[numeric_val].clip(lower=-1e6, upper=1e6)  # Replace extremely large values with a suitable range
    X[numeric_val].fillna(X[numeric_val].mean(), inplace=True)  # Replace NaN values with column mean


    2. Scale all of the values in X[numeric_col] by using the method scaler.fit_transform(__) and assign it to the variable X[numerical_col]

10. We now want to turn the categorical columns into a machine readable format so it can be interpretted by our model, we do this using the Label encoder object

    1. Assign a variable called le to the object LabelEncoder()

    2. Assign a list of the non-numerical categorical columns to a variable called 'cat_col'

    3. use a for loop to re-assign each categorical column X[cols] to its label encoded version by using le.fit_transform(X[cols])

11. We also want to do the same thing to our target variable

    1. Do the same thing to our variable y:
       a. Assign the object LabelEncoder() to le

        b. Use fit_transform to change the variable y

12. We are then splitting our training dataset into train and a test datasets, this will allow us to evaluate which model is best for predicting, we also cannot use the actual test dataset because this results in data leakage, data leakage occurs when information from the testing set is inadvertently used during model training. This can lead to overly optimistic performance estimates

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.2, random_state = 42, stratify = y)

print(f"X_train shape: ", X_train.shape)
print(f"X_val shape: ", X_val.shape)

#### **🛩 Modelling**
****

    1. fill in the gaps

- A________: The overall correctness of a classification model in predicting the correct labels.

- F1 Score: A measure of a model's accuracy that considers both precision and recall, providing a b_________ assessment.

- Precision: The proportion of true positive predictions (correctly predicted positives) out of all positive predictions made.

- Recall: The proportion of true positive predictions (correctly predicted positives) out of all actual positive instances.

- AUC (Area Under the Curve): The measure of a model's performance by calculating the area under the receiver operating characteristic (ROC) curve.

    2. Assign the object 'LogisticRegression()' to the variable lr

    3. Use the lr.fit() method to train the algorithm on the X_train and y_train lists

    4. Use the lr.predict() method to predict the values of X_val and assign this to a variable called y_pred

In [ ]:
accuracy = accuracy_score(y_val, y_pred)
print("Accuracy:", accuracy)

auc_score = roc_auc_score(y_val, y_pred)
print("AUC Score:", auc_score)


F1_score = f1_score(y_val, y_pred)
print("F1 Score:", F1_score)

Recall_score = recall_score(y_val, y_pred)
print("Recall Score:", Recall_score)

Precision_score = precision_score(y_val, y_pred)
print("Precision Score:", Precision_score)



#### **🛩 Test prediction**
****

15. we now implement all of our necessary changes to the test dataset that we had previously done to the train dataset as follows


In [ ]:
test.head()

**a.** Taking the natural logarithm of the numerical variables that are skewed, the is done to make it more suitable for model predicting and reduce skewedness

    1. Take the natural logarithm of the 2 continuous numerical variables (hint: You previously did this to the train dataset)

**d.** Define the columns for our x and y

    2.Define the X and Y variables (hint: x is all categories apart from the id and the one that we are trying to predict and the y variables is only the column that we are trying to predict)

**e.** We now normalise the numerical variables

    1. Assign a variable named scaler to the object 'StandardScaler()'

In [ ]:

numeric_col = ["Age", "Flight Distance", "Departure Delay in Minutes"]

# Handle the problematic values in X
for numeric_val in numeric_col:
    X[numeric_val] = X[numeric_val].replace([np.inf, -np.inf], np.nan)
    X[numeric_val] = X[numeric_val].clip(lower=-1e6, upper=1e6)  # Replace extremely large values with a suitable range
    X[numeric_val].fillna(X[numeric_val].mean(), inplace=True)  # Replace NaN values with column mean

X[numeric_col] = scaler.fit_transform(X[numeric_col])

**f.** We now encode our categorical variables

    1. Assign a variable called le to the object LabelEncoder()

    2. Assign a list of the non-numerical categorical columns to a variable called 'cat_col'

    3. use a for loop to re-assign each categorical column X[cols] to its label encoded version by using le.fit_transform(X[cols])

    1. Do the same thing to our variable y:
       a. Assign the object LabelEncoder() to le

        b. Use fit_transform to change the variable y

16. Lastly we used our pretrained model to predict whetehr airline passengers were satisfied or not from the test dataset and retreive the key metrics for our model

    1. Now we use the lr.predict() method to predict y using X. Assign the output of this to a variable called y_pred

In [ ]:
final_sub = pd.DataFrame({'id': test['id'], 'satisfaction': y_pred})

In [ ]:
final_sub.to_csv('submission.csv', index=False)